In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from copy import copy

In [10]:
# Read recipe inputs
yf1985_MFI = dataiku.Dataset("yf1985_MFI")
df_mfi = yf1985_MFI.get_dataframe()

In [12]:
# Va prendre toutes les colonnes MFI, puis MFI_MA
colonnes_mfi = df_mfi.columns[6:].to_list()

In [14]:
# Périodes utilisées pour les comparaisons de variation de valeur dans le temps, en jours
periodes = [3,5,8]

In [15]:
for colonne in colonnes_mfi : 
    
    for p in periodes :
        '''Calcul des variations (Slope)
        d'abord MFI valeur absolue, à -3, -5 et -8 jours
        puis pour pour les SMA sur les périodes définies en amont de la boucle'''
        
        col_name_var = "var_" + str(colonne) + "_" + str(p) + "j"
        df_mfi[col_name_var] = round( (df_mfi[colonne] - df_mfi[colonne].shift(p)) / df_mfi[colonne].shift(p) ,2)

In [18]:
origine = df_mfi.shape[0]

In [20]:
df_mfi.dropna(how='any', inplace=True)
df_mfi.reset_index(drop=True, inplace=True)

In [25]:
fin = df_mfi.shape[0]
delta = origine-fin

In [26]:
# Write recipe outputs
yf1985_MFI_enriched = dataiku.Dataset("yf1985_MFI_enriched")

if delta == periodes[-1] and df_mfi.isnull().sum().sum() == 0:
    print("Prêt à écrire. Test OK. Nb lignes supprimées : ",delta)
    yf1985_MFI_enriched.write_with_schema(df_mfi)
else:
    print("Erreur de vérification, écriture annulée")

Prêt à écrire. Test OK. Nb lignes supprimées :  8
9205 rows successfully written (hvmJZXks8J)
